In [1]:
import pandas as pd

In [13]:
df_history = pd.read_csv("../data/processed/stocks-history.csv", parse_dates=["DATE"])
df_dividends = pd.read_csv("../data/processed/stocks-dividends.csv", parse_dates=["DATE"])

In [9]:
df_history.head()

,DATE,CD_CVM,TICKER,PRICE,PL
0,2014-10-06,906.0,BBDC4,8.793426,NaN
1,2014-10-07,906.0,BBDC4,8.735580,NaN
2,2014-10-08,906.0,BBDC4,8.661528,NaN
3,2014-10-09,906.0,BBDC4,8.830453,NaN
4,2014-10-10,906.0,BBDC4,8.388464,NaN


In [10]:
df_dividends.head()

,CD_CVM,TICKER,DATE,VALUE,TYPE
0,906,BBDC4,2024-12-02,0.018975,JCP
1,906,BBDC4,2024-11-01,0.018975,JCP
2,906,BBDC4,2024-10-01,0.018975,JCP
3,906,BBDC4,2024-09-30,0.197528,JCP
4,906,BBDC4,2024-09-02,0.018975,JCP


In [50]:
df_test = pd.merge(df_history, df_dividends.groupby(["TICKER", "DATE"])["VALUE"].sum(), how="left", on=["TICKER", "DATE"])
df_test["VALUE"] = df_test["VALUE"].fillna(0)

df_dividends_rolling = df_test.groupby("TICKER").rolling(window="365D", on="DATE")["VALUE"].sum()
df_dividends_rolling = df_dividends_rolling.reset_index()
df_dividends_rolling = df_dividends_rolling.rename(columns={"VALUE": "Dividends_1y"})

df_test = pd.merge(df_test, df_dividends_rolling, how="left", on=["TICKER", "DATE"])

df_test["DIVIDEND_YIELD"] = df_test["Dividends_1y"] / df_test["PRICE"]

df_test = df_test.drop(["VALUE", "Dividends_1y"], axis=1)

df_test

,DATE,CD_CVM,TICKER,PRICE,PL,DIVIDEND_YIELD
0,2014-10-06,906.0,BBDC4,8.793426,NaN,0.000000
1,2014-10-07,906.0,BBDC4,8.735580,NaN,0.000000
2,2014-10-08,906.0,BBDC4,8.661528,NaN,0.000000
3,2014-10-09,906.0,BBDC4,8.830453,NaN,0.000000
4,2014-10-10,906.0,BBDC4,8.388464,NaN,0.000000
...,...,...,...,...,...,...
7299,2024-09-30,1023.0,BBAS3,27.180000,4.457019,0.092655
7300,2024-10-01,1023.0,BBAS3,27.080000,4.440621,0.092997
7301,2024-10-02,1023.0,BBAS3,27.150000,4.452099,0.092757
7302,2024-10-03,1023.0,BBAS3,26.900000,4.411104,0.093619
